In [19]:
import numpy as np
from itertools import product
import neal

def delta(i, j):
    if i == j:
        return 1
    else:
        return 0
    
class SVM:
    def __init__(self, B, K, C, gamma, xi, N, sampler_type='SA'):
        self.B = B
        self.K = K
        self.C = C
        self.gamma = gamma
        self.xi = xi
        self.N = N
        self.sampler_type = sampler_type
        self.sampler = neal.SimulatedAnnealingSampler()
        self.data = None  # Will hold the training data
        self.t = None  # Will hold the training labels
        self.alpha = None
        self.b = None
    
    def kernel(self, x, y):
        # Using the Gaussian kernel
        return np.exp(-self.gamma * np.linalg.norm(x-y)**2)

    def train_SVM(self, data, t):
        self.data = data
        self.t = t
        # Create the QUBO for the SVM
        Q_tilde = np.zeros((self.K*self.N, self.K*self.N))
        for n in range(self.N):
            for m in range(self.N):
                for k in range(self.K):
                    for j in range(self.K):
                        Q_tilde[(self.K*n+k, self.K*m+j)] = 0.5*(self.B**(k+j))*t[n]*t[m] * \
                            (self.kernel(data[n], data[m])+self.xi) - \
                            (delta(n, m)*delta(k, j)*(self.B**k))

        Q = np.zeros((self.K*self.N, self.K*self.N))
        for j in range(self.K*self.N):
            Q[(j, j)] = Q_tilde[(j, j)]
            for i in range(self.K*self.N):
                if i < j:
                    Q[(i, j)] = Q_tilde[(i, j)] + Q_tilde[(j, i)]

        size_of_q = Q.shape[0]
        qubo = {(i, j): Q[i, j]
                for i, j in product(range(size_of_q), range(size_of_q))}

        # Sample from QUBO using the selected sampler
        response = self.sampler.sample_qubo(qubo, num_reads=100)

        # Decode the response
        sample = response.first.sample
        alpha = np.array([sum((self.B**k) * sample[self.K * n + k] for k in range(self.K)) for n in range(self.N)])
        b = np.mean([t[n] - sum(alpha[m] * t[m] * self.kernel(data[m], data[n]) for m in range(self.N)) for n in range(self.N)])

        self.alpha = alpha
        self.b = b
        
        return alpha, b

    def predict_class(self, x, alpha, b):
        # Use stored data and labels for prediction
        f = sum(alpha[i] * self.t[i] * self.kernel(self.data[i], x) for i in range(self.N)) + b
        # return 1 if f > 0 else -1
        return f

In [20]:
import numpy as np
from pyqubo import Array, Binary, Placeholder, Constraint, solve_qubo
from itertools import combinations
from sklearn.metrics import accuracy_score
import math
import neal
from collections import Counter

class OneVsRestClassifier:
    def __init__(self, class_num, classifier, params=None):
        """binary classifierを受け取り、クラスの数分インスタンスを作成する."""
        self.class_num = class_num
        self.alpha = None
        self.b = None

        if params is None:
            self.classifiers = [classifier() for _ in range(class_num)]
        else:
            self.classifiers = [classifier(**params) for _ in range(class_num)]

    def solve(self, x, y):
        for i in range(self.class_num):
            print(f"Training classifier {i}...")
            alpha, b = self.classifiers[i].train_SVM(x, self.re_labaling(y, i))
            self.alpha = alpha
            self.b = b
        return self

    def re_labaling(self, y, pos_label):
        """labelを受け取り、pos_labelに指定したカテゴリを+1、それ以外を-1にラベリングしなおしたデータを返す."""
        return np.where(y == pos_label, 1, -1)

    def argmax_by_E(self, result):
        for i in range(result.shape[0]):
            for j in range(result.shape[1]):
                if np.sum(result[:, j], axis=0) == -1: #case (1,-1,-1)
                    pass
                elif np.sum(result[:, j], axis=0) == 1: #case (1,1,-1)
                    a = np.array(np.where(result[:, j] == 1))[0][0]
                    b = np.array(np.where(result[:, j] == 1))[0][1]
                    if self.classifiers[a].energy > self.classifiers[b].energy:
                        result[a, j] = -1
                    else:
                        result[b, j] = -1
                elif np.sum(result[:, j], axis=0) == 3: #case (1,1,1)
                    min_e = np.argmin(np.array([self.classifiers[0].energy, self.classifiers[1].energy, self.classifiers[2].energy]))
                    result[0:min_e, j] = -1
                    result[min_e:i, j] = -1
                elif np.sum(result[:, j], axis=0) == -3: #case (-1,-1,-1)
                    min_e = np.argmin(
                        np.array([self.classifiers[0].energy, self.classifiers[1].energy, self.classifiers[2].energy]))
                    result[min_e, j] = 1

        print("result", result)
        return np.argmax(result, axis=0)

    def predict(self, X):
        result = np.array([model.predict_class(X) for model in self.classifiers])
        #print("result", result)
        #print("result type and shape", type(result), result.shape)
        #result type and shape <class 'numpy.ndarray'> (3, 150)
        return self.argmax_by_E(result)
    
    def compute_metrics(_SVM,alpha,data,t,b,N,validation_pts):
        tp, fp, tn, fn = 0, 0, 0, 0
        for i in range(N, N+validation_pts):
            predicted_cls = _SVM.predict_class(data[i], alpha, b)
            y_i = t[i]
            if(y_i == 1):
                if(predicted_cls > 0):
                    tp += 1
                else:
                    fp += 1
            else:
                if(predicted_cls < 0):
                    tn += 1
                else:
                    fn += 1

        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f_score = tp/(tp + 1/2*(fp+fn))
        accuracy = (tp + tn)/(tp+tn+fp+fn)

        return precision,recall,f_score,accuracy

    def evaluate(self, X, y):
        precision,recall,f_score,accuracy = utils.compute_metrics(_SVM,self.alpha,data,t,b,args['trainingPoints'],args['validationPoints'])
        # pred = self.predict(X)
        # print("pred result",pred)
        # return accuracy_score(y, pred)
        
        print(f'{precision=} {recall=} {f_score=} {accuracy=}')
        return precision
    
def qubo_dict_to_np(qubo_dict):
    n = int(np.sqrt(len(qubo_dict)))
    qubo_np = np.zeros((n, n))
    for ((i, j), value) in qubo_dict.items():
        qubo_np[int(i.split('[')[1].split(']')[0])][int(j.split('[')[1].split(']')[0])] = value
    return qubo_np

In [21]:
class OneVsRestClassifier:
    def __init__(self, class_num, classifier, params):
        self.class_num = class_num
        self.classifiers = [classifier(**params) for _ in range(class_num)]

    def re_labeling(self, y, pos_label):
        """Re-labels the data for the binary classification task: +1 for pos_label, -1 otherwise."""
        return np.where(y == pos_label, 1, -1)

    def solve(self, x, y):
        for i in range(self.class_num):
            print(f"Training classifier for class {i}...")
            binary_labels = self.re_labeling(y, i)
            self.classifiers[i].train_SVM(x, binary_labels)

    def predict(self, X):
        """Predicts classes by selecting the classifier with the highest decision function value."""
        predictions = np.array([clf.predict_class(X, clf.alpha, clf.b) for clf in self.classifiers]).T
        print(predictions)
        return np.argmax(predictions, axis=1)

import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load blob dataset
n_samples = 3 * 20
X, y = datasets.make_blobs(n_samples=n_samples, centers=3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

# Parameters for the SVM in the OneVsRest setup
params = {
    "B": 2,
    "K": 2,
    "C": 3,
    "gamma": 0.5,
    "xi": 1,
    "N": len(X_train),
    "sampler_type": "SA"  # Using simulated annealing sampler
}

# Initialize the multiclassifier with the OneVsRest strategy
one_vs_rest = OneVsRestClassifier(class_num=3, classifier=SVM, params=params)
one_vs_rest.solve(X_train, y_train)

# Predictions on the test set
y_pred = one_vs_rest.predict(X_test)

# Evaluate performance
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

Training classifier for class 0...
Training classifier for class 1...
Training classifier for class 2...
[-0.36668102 -0.33741457 -0.43433639]


AxisError: axis 1 is out of bounds for array of dimension 1